# Linear Mixed Models

Build some linear mixed models about our data

## Import data

In [1]:
import os
import sys
from socket import gethostname

hostname = gethostname()

import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
from matplotlib.colors import ListedColormap
import pandas as pd
import seaborn as sns
import json

from itertools import cycle

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OrdinalEncoder
from sklearn.inspection import permutation_importance

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error

from scipy.stats import linregress

import statsmodels.api as sm
import statsmodels.formula.api as smf

# there is a FutureWarning in sklearn StandardScalar which is really annoying. This ignores it.
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

try:
  import google.colab
  IN_COLAB = True
  !pip install adjustText
  from google.colab import drive
  drive.mount('/content/drive')
  datadir = '/content/drive/MyDrive/Projects/CF/Adelaide/CF_Data_Analysis'
except ImportError:
  IN_COLAB = False
  datadir = '..'


if hostname.startswith('hpc-node'):
    IN_DEEPTHOUGHT = True
    sys.path.append('..')
else:
    IN_DEEPTHOUGHT = False

import cf_analysis_lib


## Read the data frames

In [2]:
sequence_type = "MGI"
datadir = '..'
#sslevel = 'level2_norm_ss.tsv.gz'
sslevel = 'subsystems_norm_ss.tsv.gz'
ss_df = cf_analysis_lib.read_subsystems(os.path.join(datadir, sequence_type, "FunctionalAnalysis", "subsystems", sslevel), sequence_type)
ss_df = ss_df.T
print(f"The subsystems df has shape: {ss_df.shape}")

taxa = "genus"
genus_otu = cf_analysis_lib.read_taxonomy(datadir, sequence_type, taxa)
genus_otu = genus_otu.T
print(f"The taxonomy df has shape: {genus_otu.shape}")
metadata = cf_analysis_lib.read_metadata(datadir, sequence_type, categorise=True)
print(f"The metadata df has shape: {metadata.shape}")

df = ss_df.merge(genus_otu, left_index=True, right_index=True, how='inner')

The subsystems df has shape: (127, 769)
The taxonomy df has shape: (127, 3581)
The metadata df has shape: (127, 166)


/home/edwa0468/GitHubs/CF_Data_Analysis/Jupyter/../cf_analysis_lib/read_data.py:125: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  metadata[c] = pd.to_datetime(metadata[c])


## Rename the columns

Statsmodels can't handle column names with spaces or special characters, and so we rename them

In [4]:
df.columns = [re.sub(r'\W+', '_', col) for col in df.columns]
metadata.columns = [re.sub(r'\W+', '_', col) for col in metadata.columns]

In [5]:
df.head(3)

,2_3_diacetamido_2_3_dideoxy_d_mannuronic_acid,2_O_alpha_mannosyl_D_glycerate_utilization,2_aminophenol_Metabolism,2_ketoacid_oxidoreductases_disambiguation,2_oxoglutarate_dehydrogenase_,2_phosphoglycolate_salvage,3_amino_5_hydroxybenzoic_Acid_Synthesis,4_hydroxybenzoyl_CoA_reductase,5_methylaminomethyl_2_thiouridine,A_Hypothetical_Protein_Related_to_Proline_Metabolism,...,Oceaniferula,Persicirhabdus,Phragmitibacter,Prosthecobacter,Roseibacillus,Roseimicrobium,Sulfuriroseicoccus,Verrucomicrobium,Eremiobacter,Methylomirabilis
1068841_20180306_S,10.085904,2784.895948,516.160945,231.778018,122.573392,342.327431,783.041862,0.0,4136.703664,827.439614,...,1.347606,0.0,0.0,0.0,0.539042,0.0,0.0,0.539042,0.0,0.0
1447437_20171212_S,59.260325,1065.438272,543.947408,554.136026,428.171446,301.915763,679.310468,0.0,3913.676651,732.956657,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
1128691_20171206_S,0.000000,426.619709,912.767750,49.606943,213.640568,277.798880,423.312580,0.0,2093.412992,236.459761,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0


In [6]:
metadata.head(3)

,minion,MGI,pwCF_ID,Sample_date,IP_vs_OP,Hospital,Room,Age,Age_groups,Paediatric_vs_Adult,...,Sum_of_meds,Sum_of_antifungals,Sum_of_steroid_mabs,DNA_extraction__conc,SAGC_ULN,DNA_Conc_ng_ul_,Index_I7,Index_I5,Mean_Size_BP,Total_Clusters_Passing_Filter_Million_
NAME,,,,,,,,,,,,,,,,,,,,,
634207_20180510_S,NaN,634207_20180510_S,634207,5/10/2018,IP,WCH,Adol Rm9,17,3,Paediatric,...,1,0,0,0.000,SAGCFN_22_01856,7.82,CGGACGATTC,CCACCACCTA,651,2.9
634207_20180517_S,NaN,634207_20180517_S,634207,5/17/2018,IP,WCH,Adol Rm9,17,3,Paediatric,...,1,0,0,0.134,SAGCFN_22_01827,22.80,AGCGATAG,CCTATCCT,633,2.4
715927_20180205_S,715927_20180205_S,715927_20180205_S,715927,2/05/2018,OP,WCH,Level 6 DK Office,13,3,Paediatric,...,1,0,0,0.326,SAGCFN_22_01797,16.50,TAATGCGC,AGGCGAAG,516,3.4


In [7]:
" ".join(map(str, list(metadata["N12M_Pseudomonas_aeruginosa"].values)))

'0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan 0.0 nan nan 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0'

## Convert categories and objects to numeric forms

In [8]:
# we make a copy of each data frame and then overwrite the columns. Internally this is more efficient than making a df one
# column at a time.
encoded_metadata = metadata.copy()
to_delete = []
for col in metadata:
    if pd.api.types.is_numeric_dtype(metadata[col]):
        continue
    elif col == sequence_type:
        continue
    elif metadata[col].dtypes == 'category':
        encoded_metadata[col] = metadata[col].cat.codes
    else:
        print(f'Dropping {col}: {metadata[col].dtypes}')
        to_delete.append(col)

encoded_metadata = encoded_metadata.drop(columns=to_delete)


Dropping minion: object
Dropping Sample_date: object
Dropping IP_vs_OP: object
Dropping Hospital: object
Dropping Room: object
Dropping Paediatric_vs_Adult: object
Dropping Gender: object
Dropping Sample_Type: object
Dropping Culture_Result_Matched_with_sequenced_sample_: object
Dropping Cutured_in_previous_12_months: object
Dropping Others_cultured: object
Dropping Spec_IgG: object
Dropping Cystic_Fibrosis_related_diabetes_CFRD_: object
Dropping Pancreatic_insufficiency_PI_: object
Dropping CF_gene_1: object
Dropping CF_gene_2: object
Dropping Notes: object
Dropping CFLD: object
Dropping Next_PA_Positive_Date: object
Dropping Positive_for_PA_within_next_12_months: object
Dropping Culture_for_next_sample: object
Dropping Antibiotics_duration_: object
Dropping Antibiotics_YN: object
Dropping SAGC_ULN: object
Dropping Index_I7: object
Dropping Index_I5: object


In [31]:
metadata['Next_PA_Positive_Date']

NAME
634207_20180510_S            NaN
634207_20180517_S            NaN
715927_20180205_S     18/03/2019
715927_20180213_S     18/03/2019
715927_20180226_S     18/03/2019
                         ...    
1651490_20180206_S     4/09/2018
1651490_20171215_S     4/09/2018
1658447_20171006_S     6/10/2017
1664053_20180406_S     6/04/2018
1690154_20180406_S     6/04/2018
Name: Next_PA_Positive_Date, Length: 127, dtype: object

In [12]:
#encoder = LabelEncoder()
encoder = OrdinalEncoder()

# we make a copy of each data frame and then overwrite the columns. Internally this is more efficient than making a df one
# column at a time.
encoded_metadata = metadata.copy()
for col in metadata:
    if not pd.api.types.is_numeric_dtype(metadata[col]):
        encoded_metadata[col] = metadata[col].cat.codes

encoded_df = df.copy()
for col in df:
    if not pd.api.types.is_numeric_dtype(df[col]):
        encoded_df[col] = encoder.fit_transform([metadata[col].values])[0]  # Ensure all values are strings



Not sure about minion: object
Not sure about MGI: object
Not sure about pwCF_ID: int64
Not sure about Sample_date: object
Not sure about Room: object
Not sure about Age: int64
Not sure about H2_Uncorrected: float64
Not sure about CH4_Uncorrected: float64
Not sure about CO2: float64
Not sure about H2_Corrected: float64
Not sure about CH4_Corrected: float64
Not sure about CH4_H2_ratio_corrected: float64
Not sure about Corr_: float64
Not sure about Culture_Result_Matched_with_sequenced_sample_: object
Not sure about Pseudomonas_Culture: float64
Not sure about Cutured_in_previous_12_months: object
Not sure about Others_cultured: object
Not sure about IgE: float64
Not sure about Spec_IgE: float64
Not sure about Spec_IgG: object
Not sure about Precipitins: float64
Not sure about FVC: float64
Not sure about FEV1: float64
Not sure about Best_FEV1: int64
Not sure about FEV1_best_FEV1: float64
Not sure about CF_gene_2: object
Not sure about Notes: object
Not sure about CS_Scedosporium_apiospermu

In [13]:
" ".join(map(str, list(encoded_metadata["N12M_Pseudomonas_aeruginosa"].values)))

'0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan 0.0 nan nan 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0'

In [16]:
metadata["N12M_Pseudomonas_aeruginosa"].dropna().astype(int)

NAME
634207_20180510_S     0
634207_20180517_S     0
715927_20180205_S     0
715927_20180213_S     0
715927_20180226_S     0
                     ..
1651490_20180206_S    1
1651490_20171215_S    1
1658447_20171006_S    1
1664053_20180406_S    1
1690154_20180406_S    1
Name: N12M_Pseudomonas_aeruginosa, Length: 110, dtype: int64

# Predict something "simple"

Here, we are just trying to predict Pseudomonas positive in the next 12 months!

In [29]:
df_combined = encoded_df.merge(encoded_metadata, left_index=True, right_index=True, how='inner')
df_combined = df_combined.dropna(subset=['N12M_Pseudomonas_aeruginosa'])
df_combined = df_combined.dropna(subset=["pwCF_ID"])
df_combined["N12M_Pseudomonas_aeruginosa"] = df_combined["N12M_Pseudomonas_aeruginosa"].astype(int)
df_combined = df_combined.reset_index()

print(f"df: {df.shape}")
print(f"metadata: {metadata.shape}")
print(f"df_combined: {df_combined.shape}")

df: (127, 4350)
metadata: (127, 166)
df_combined: (110, 4517)


In [47]:
random_column_number = 100
#random_columns = np.random.choice(df.columns, size=random_column_number, replace=False)
random_columns = ['Pedobacter']
predictors = " + ".join(random_columns)  # Concatenate all bacterial abundance column names
formula = f"N12M_Pseudomonas_aeruginosa ~ {predictors} + CS_Pseudomonas_aeruginosa + Pseudomonas_Culture"
print(formula)

N12M_Pseudomonas_aeruginosa ~ Pedobacter + CS_Pseudomonas_aeruginosa + Pseudomonas_Culture


In [27]:
print(set(formula.split(" ~ ")[1].replace("+", "").split()))

{'Limosilactobacillus', 'Pedobacter', 'CS_Pseudomonas_aeruginosa', 'Pseudomonas_Culture'}


In [48]:
genus_otu.columns

Index(['Aegiribacteria', 'Kapaibacterium', 'Stahlbacteria', 'Vermiphilus',
       'Babela', 'Chromulinivorax', 'Ozemobacter', 'Desulfacyla',
       'Dissulfurimicrobium', 'Dissulfurirhabdus',
       ...
       'Oceaniferula', 'Persicirhabdus', 'Phragmitibacter', 'Prosthecobacter',
       'Roseibacillus', 'Roseimicrobium', 'Sulfuriroseicoccus',
       'Verrucomicrobium', 'Eremiobacter', 'Methylomirabilis'],
      dtype='object', name='taxonomy', length=3581)

In [36]:
df_combined.dropna(subset=random_columns).shape

(110, 4517)

In [38]:
all_columns = random_columns + ['CS_Pseudomonas_aeruginosa' ,'Pseudomonas_Culture', 'N12M_Pseudomonas_aeruginosa']
all_columns

['Pedobacter',
 'CS_Pseudomonas_aeruginosa',
 'Pseudomonas_Culture',
 'N12M_Pseudomonas_aeruginosa']

In [41]:
df_combined

,index,2_3_diacetamido_2_3_dideoxy_d_mannuronic_acid,2_O_alpha_mannosyl_D_glycerate_utilization,2_aminophenol_Metabolism,2_ketoacid_oxidoreductases_disambiguation,2_oxoglutarate_dehydrogenase_,2_phosphoglycolate_salvage,3_amino_5_hydroxybenzoic_Acid_Synthesis,4_hydroxybenzoyl_CoA_reductase,5_methylaminomethyl_2_thiouridine,...,Sum_of_meds,Sum_of_antifungals,Sum_of_steroid_mabs,DNA_extraction__conc,SAGC_ULN,DNA_Conc_ng_ul_,Index_I7,Index_I5,Mean_Size_BP,Total_Clusters_Passing_Filter_Million_
0,1068841_20180306_S,10.085904,2784.895948,516.160945,231.778018,122.573392,342.327431,783.041862,0.0,4136.703664,...,0.0,0.0,0.0,1.070,0.0,42.800,0.0,0.0,417,2.8
1,1447437_20171212_S,59.260325,1065.438272,543.947408,554.136026,428.171446,301.915763,679.310468,0.0,3913.676651,...,0.0,0.0,0.0,1.510,0.0,32.600,0.0,0.0,498,4.7
2,1128691_20171206_S,0.000000,426.619709,912.767750,49.606943,213.640568,277.798880,423.312580,0.0,2093.412992,...,0.0,0.0,0.0,1.750,0.0,33.400,0.0,0.0,535,4.5
3,1128691_20171218_S,0.000000,659.087578,864.137047,139.140711,235.318676,355.174973,538.254855,0.0,2050.494687,...,0.0,0.0,0.0,3.400,0.0,30.800,0.0,0.0,352,4.8
4,1128691_20180116_S,14.478968,159.268644,593.637673,48.263225,358.354449,202.705547,348.701804,0.0,1570.967988,...,0.0,0.0,0.0,0.708,0.0,27.400,0.0,0.0,500,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,877469_20190115_S,6.849038,4314.893843,1041.053753,22.830126,67.120571,352.725449,402.951727,0.0,4232.705389,...,0.0,0.0,0.0,0.000,0.0,0.332,0.0,0.0,383,0.2
106,892355_20180123_S,11.073270,227.002043,586.883332,606.261555,683.036230,243.611949,653.784340,0.0,4210.611072,...,0.0,0.0,0.0,2.890,0.0,21.200,0.0,0.0,349,5.1
107,895293_20180502_S,11.343610,250.504720,1278.046721,264.054032,86.589556,229.708102,753.089661,0.0,3424.824904,...,0.0,0.0,0.0,0.442,0.0,16.900,0.0,0.0,584,4.6
108,980574_20180403_S,15.281074,1205.624053,444.731949,383.080719,119.209942,228.425711,929.423029,0.0,3243.276243,...,0.0,0.0,0.0,0.216,0.0,34.600,0.0,0.0,364,2.2


### Check for colinearity

If VIF > 10, the variable is highly collinear. Remove or combine collinear predictors.

In [42]:
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [44]:
X = df_combined[all_columns]
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)


                      Variable       VIF
0                   Pedobacter  1.004945
1    CS_Pseudomonas_aeruginosa       NaN
2          Pseudomonas_Culture  1.004945
3  N12M_Pseudomonas_aeruginosa       NaN


/home/edwa0468/miniconda3/envs/bioinformatics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1738: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


In [53]:
print(df_combined["pwCF_ID"].nunique())

56


In [56]:
df_combined.nunique() > 1

index                                             True
2_3_diacetamido_2_3_dideoxy_d_mannuronic_acid     True
2_O_alpha_mannosyl_D_glycerate_utilization        True
2_aminophenol_Metabolism                          True
2_ketoacid_oxidoreductases_disambiguation         True
                                                 ...  
DNA_Conc_ng_ul_                                   True
Index_I7                                         False
Index_I5                                         False
Mean_Size_BP                                      True
Total_Clusters_Passing_Filter_Million_            True
Length: 4517, dtype: bool

In [60]:
df_combined3 = df_combined.loc[:, df_combined.nunique() > 1].dropna()
print(set(df.columns) - set(df_combined3.columns))

{'Rhodonellum', 'Fluviicoccus', 'Pseudotabrizicola', 'Halocynthiibacter', 'Haematobacter', 'DNA_phosphorothioation_dependent_restriction_system_DptFGH', 'Albibacterium', 'Geminisphaera', 'Spirillospora', 'Neptunochlamydia', 'Georhizobium', 'Brevefilum', 'Rhodobaca', 'Lasso_peptide_maturation_cluster', 'Vitiosangium', 'Geopsychrobacter', 'Chroococcus', 'Hephaestia', 'Mycobacterial_cell_wall_virulence_lipid_phthiocerol_dimycocerosate_PDIM_', 'Sporosalibacterium', 'Inmirania', 'Hongsoonwoonella', 'Luteithermobacter', 'Emcibacter', 'Methylohalobius', 'Salibaculum', 'Hydrogenobacter', 'Toluene_4_monooxygenase_T4MO_', 'Igneacidithiobacillus'}


In [64]:
df_combined["N12M_Pseudomonas_aeruginosa"].astype(int).values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## Use PCA to reduce the dimensionality of the data

In [50]:
from sklearn.decomposition import PCA

# Apply PCA to reduce bacterial abundance columns to 5 components
pca = PCA(n_components=5)
pca_components = pca.fit_transform(df_combined[df.columns])  # Add bacterial abundance columns
pca_df = pd.DataFrame(pca_components, columns=["PC1", "PC2", "PC3", "PC4", "PC5"])

# Add PCA components to the combined DataFrame
df_combined2 = pd.concat([df_combined.reset_index(drop=True), pca_df], axis=1)
df_combined2

,index,2_3_diacetamido_2_3_dideoxy_d_mannuronic_acid,2_O_alpha_mannosyl_D_glycerate_utilization,2_aminophenol_Metabolism,2_ketoacid_oxidoreductases_disambiguation,2_oxoglutarate_dehydrogenase_,2_phosphoglycolate_salvage,3_amino_5_hydroxybenzoic_Acid_Synthesis,4_hydroxybenzoyl_CoA_reductase,5_methylaminomethyl_2_thiouridine,...,DNA_Conc_ng_ul_,Index_I7,Index_I5,Mean_Size_BP,Total_Clusters_Passing_Filter_Million_,PC1,PC2,PC3,PC4,PC5
0,1068841_20180306_S,10.085904,2784.895948,516.160945,231.778018,122.573392,342.327431,783.041862,0.0,4136.703664,...,42.800,0.0,0.0,417,2.8,105232.294537,29481.534204,3009.236199,-748.874391,1393.844533
1,1447437_20171212_S,59.260325,1065.438272,543.947408,554.136026,428.171446,301.915763,679.310468,0.0,3913.676651,...,32.600,0.0,0.0,498,4.7,-68177.360588,-173244.331191,150955.115109,58275.741094,15805.072273
2,1128691_20171206_S,0.000000,426.619709,912.767750,49.606943,213.640568,277.798880,423.312580,0.0,2093.412992,...,33.400,0.0,0.0,535,4.5,-211025.239559,-151897.294836,-320966.178592,262834.328944,74715.555125
3,1128691_20171218_S,0.000000,659.087578,864.137047,139.140711,235.318676,355.174973,538.254855,0.0,2050.494687,...,30.800,0.0,0.0,352,4.8,-188035.291051,-141834.133045,-266846.807833,226853.551490,69220.178950
4,1128691_20180116_S,14.478968,159.268644,593.637673,48.263225,358.354449,202.705547,348.701804,0.0,1570.967988,...,27.400,0.0,0.0,500,3.8,-231769.880818,-155931.482253,-328612.917308,256193.671691,79085.256264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,877469_20190115_S,6.849038,4314.893843,1041.053753,22.830126,67.120571,352.725449,402.951727,0.0,4232.705389,...,0.332,0.0,0.0,383,0.2,433871.879019,-50971.981671,-5320.962053,24773.058456,24120.785059
104,892355_20180123_S,11.073270,227.002043,586.883332,606.261555,683.036230,243.611949,653.784340,0.0,4210.611072,...,21.200,0.0,0.0,349,5.1,-242787.205324,-340573.238777,401656.123059,192192.110121,-6890.326181
105,895293_20180502_S,11.343610,250.504720,1278.046721,264.054032,86.589556,229.708102,753.089661,0.0,3424.824904,...,16.900,0.0,0.0,584,4.6,95694.320998,22992.264925,-6039.669020,-3148.567876,13543.206780
106,980574_20180403_S,15.281074,1205.624053,444.731949,383.080719,119.209942,228.425711,929.423029,0.0,3243.276243,...,34.600,0.0,0.0,364,2.2,-4008.324996,95743.541072,17895.588170,2995.813400,23541.421539


In [57]:
formula = "N12M_Pseudomonas_aeruginosa ~ Pedobacter + CS_Pseudomonas_aeruginosa + Pseudomonas_Culture"
# Fit the GLMM
model = smf.mixedlm(
    formula=formula,
    data=df_combined3,
    groups=df_combined3["pwCF_ID"],  # Grouping by pwCF_ID for random effects
)
#family=sm.families.Binomial()  # Logistic link function
result = model.fit()

# Print the model summary
print(result.summary())

PatsyError: Error evaluating factor: NameError: name 'N12M_Pseudomonas_aeruginosa' is not defined
    N12M_Pseudomonas_aeruginosa ~ Pedobacter + CS_Pseudomonas_aeruginosa + Pseudomonas_Culture
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [40]:
df_combined = df_combined.dropna(subset=all_columns)
# Fit the GLMM
model = smf.mixedlm(
    formula=formula,
    data=df_combined,
    groups=df_combined["pwCF_ID"],  # Grouping by pwCF_ID for random effects
)
#family=sm.families.Binomial()  # Logistic link function
result = model.fit()

# Print the model summary
print(result.summary())

LinAlgError: Singular matrix

In [91]:
metadata["N12M_Pseudomonas_aeruginosa"]

NAME
634207_20180510_S     0.0
634207_20180517_S     0.0
715927_20180205_S     0.0
715927_20180213_S     0.0
715927_20180226_S     0.0
                     ... 
1651490_20180206_S    1.0
1651490_20171215_S    1.0
1658447_20171006_S    1.0
1664053_20180406_S    1.0
1690154_20180406_S    1.0
Name: N12M_Pseudomonas_aeruginosa, Length: 127, dtype: category
Categories (2, float64): [0.0, 1.0]

In [23]:
metadata["N12M_Pseudomonas_aeruginosa"].astype(int)

NAME
634207_20180510_S     0
634207_20180517_S     0
715927_20180205_S     0
715927_20180213_S     0
715927_20180226_S     0
                     ..
1651490_20180206_S    1
1651490_20171215_S    1
1658447_20171006_S    1
1664053_20180406_S    1
1690154_20180406_S    1
Name: N12M_Pseudomonas_aeruginosa, Length: 127, dtype: int64

# Early trial

Ignore the stuff down here, its probably wrong :)

In [3]:
metadata

,minion,MGI,pwCF_ID,Sample date,IP vs OP,Hospital,Room,Age,Age groups,Paediatric vs Adult,...,Sum of meds,Sum of antifungals,Sum of steroid + mabs,DNA_extraction_ conc,SAGC ULN,DNA Conc. (ng/ul),Index I7,Index I5,Mean_Size_BP,Total Clusters Passing Filter (Million)
NAME,,,,,,,,,,,,,,,,,,,,,
634207_20180510_S,NaN,634207_20180510_S,634207,5/10/2018,IP,WCH,Adol Rm9,17,3,Paediatric,...,1,0,0,0.000,SAGCFN_22_01856,7.82,CGGACGATTC,CCACCACCTA,651,2.9
634207_20180517_S,NaN,634207_20180517_S,634207,5/17/2018,IP,WCH,Adol Rm9,17,3,Paediatric,...,1,0,0,0.134,SAGCFN_22_01827,22.80,AGCGATAG,CCTATCCT,633,2.4
715927_20180205_S,715927_20180205_S,715927_20180205_S,715927,2/05/2018,OP,WCH,Level 6 DK Office,13,3,Paediatric,...,1,0,0,0.326,SAGCFN_22_01797,16.50,TAATGCGC,AGGCGAAG,516,3.4
715927_20180213_S,NaN,715927_20180213_S,715927,2/13/2018,IP,WCH,Adol Room 11,13,3,Paediatric,...,3,0,0,0.234,SAGCFN_22_01811,31.00,TCCGCGAA,CCTATCCT,443,2.7
715927_20180226_S,NaN,715927_20180226_S,715927,2/26/2018,OP,WCH,OPD 8,13,3,Paediatric,...,2,0,0,0.108,SAGCFN_22_01833,15.10,TAACTTGGTC,GATTCACGAC,510,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651490_20180206_S,1651490_20180206_S,1651490_20180206_S,1651490,2/06/2018,OP,RAH,Chest Clinic 1,27,5,Adult,...,1,0,0,4.760,SAGCFN_22_01741,26.20,ATTACTCG,AGGCGAAG,507,6.4
1651490_20171215_S,1651490_20171215_S,1651490_20171215_S,1651490,12/15/2017,OP,RAH,Chest Clinic 4,26,5,Adult,...,1,0,0,7.760,SAGCFN_22_01738,34.20,ATTACTCG,ATAGAGGC,564,6.0
1658447_20171006_S,NaN,1658447_20171006_S,1658447,10/06/2017,OP,RAH,Chest Clinic 3,26,5,Adult,...,1,0,0,0.098,SAGCFN_22_01837,13.10,CAGCAGGTCA,TACCTAAGTG,576,2.9


df: (127, 4350)
metadata: (127, 166)
df_combined: (127, 4516)


In [4]:
df_long = df.reset_index().melt(id_vars='index', var_name='taxa_functions', value_name='abundance')
df_long.rename(columns={'index': 'sample_id'}, inplace=True)
df_long

,sample_id,taxa_functions,abundance
0,1068841_20180306_S,"2,3-diacetamido-2,3-dideoxy-d-mannuronic acid",10.085904
1,1447437_20171212_S,"2,3-diacetamido-2,3-dideoxy-d-mannuronic acid",59.260325
2,1128691_20171206_S,"2,3-diacetamido-2,3-dideoxy-d-mannuronic acid",0.000000
3,1128691_20171218_S,"2,3-diacetamido-2,3-dideoxy-d-mannuronic acid",0.000000
4,1128691_20180116_S,"2,3-diacetamido-2,3-dideoxy-d-mannuronic acid",14.478968
...,...,...,...
552445,895293_20180502_S,Methylomirabilis,0.000000
552446,896213_20180427_S,Methylomirabilis,0.000000
552447,913873_20180417_S,Methylomirabilis,0.000000
552448,980574_20180403_S,Methylomirabilis,0.000000


In [5]:
merged_data =  pd.merge(df_long, metadata, left_on='sample_id', right_on=sequence_type)
merged_data.head()

,sample_id,taxa_functions,abundance,minion,MGI,pwCF_ID,Sample date,IP vs OP,Hospital,Room,...,Sum of meds,Sum of antifungals,Sum of steroid + mabs,DNA_extraction_ conc,SAGC ULN,DNA Conc. (ng/ul),Index I7,Index I5,Mean_Size_BP,Total Clusters Passing Filter (Million)
0,1068841_20180306_S,"2,3-diacetamido-2,3-dideoxy-d-mannuronic acid",10.085904,1068841_20180306_S,1068841_20180306_S,1068841,3/06/2018,OP,RAH,Chest Clinic 7,...,0,0,0,1.07,SAGCFN_22_01754,42.8,CGCTCATT,ATAGAGGC,417,2.8
1,1068841_20180306_S,2-O-alpha-mannosyl-D-glycerate utilization,2784.895948,1068841_20180306_S,1068841_20180306_S,1068841,3/06/2018,OP,RAH,Chest Clinic 7,...,0,0,0,1.07,SAGCFN_22_01754,42.8,CGCTCATT,ATAGAGGC,417,2.8
2,1068841_20180306_S,2-aminophenol Metabolism,516.160945,1068841_20180306_S,1068841_20180306_S,1068841,3/06/2018,OP,RAH,Chest Clinic 7,...,0,0,0,1.07,SAGCFN_22_01754,42.8,CGCTCATT,ATAGAGGC,417,2.8
3,1068841_20180306_S,2-ketoacid oxidoreductases disambiguation,231.778018,1068841_20180306_S,1068841_20180306_S,1068841,3/06/2018,OP,RAH,Chest Clinic 7,...,0,0,0,1.07,SAGCFN_22_01754,42.8,CGCTCATT,ATAGAGGC,417,2.8
4,1068841_20180306_S,2-oxoglutarate dehydrogenase,122.573392,1068841_20180306_S,1068841_20180306_S,1068841,3/06/2018,OP,RAH,Chest Clinic 7,...,0,0,0,1.07,SAGCFN_22_01754,42.8,CGCTCATT,ATAGAGGC,417,2.8


In [12]:
columns_no_spaces = {}
for c in merged_data.columns:
    if ' ' in c:
        columns_no_spaces[c] = c.replace(' ', '_')
        
merged_data.rename(columns = columns_no_spaces, inplace=True)


df_combined.rename(columns = columns_no_spaces, inplace=True)


In [13]:
list(df_combined.columns)

['2,3-diacetamido-2,3-dideoxy-d-mannuronic acid',
 '2-O-alpha-mannosyl-D-glycerate utilization',
 '2-aminophenol Metabolism',
 '2-ketoacid oxidoreductases disambiguation',
 '2-oxoglutarate dehydrogenase ',
 '2-phosphoglycolate salvage',
 '3-amino-5-hydroxybenzoic Acid Synthesis',
 '4-hydroxybenzoyl-CoA reductase',
 '5-methylaminomethyl-2-thiouridine',
 'A Hypothetical Protein Related to Proline Metabolism',
 'A new toxin - antitoxin system',
 'ABC transporter YeiABEF',
 'ABC transporter YxeMNO',
 'ABC transporter of unknown substrate X',
 'ABC transporter tungstate (TC 3.A.1.6.2)',
 'ABC-type iron transport system',
 'AMP to 3-phosphoglycerate',
 'ATP-dependent Nuclease',
 'ATP-dependent RNA helicases, bacterial',
 'AaeAB efflux system for hydroxylated, aromatic carboxylic acids',
 'Accessory colonization factor',
 'Acetoin, butanediol metabolism',
 'Acetolactate synthase subunits',
 'Acetophenone carboxylase 1',
 'Acetyl-CoA Pathway Wood-Ljungdahl',
 'Actinobacterial signal transducti

In [ ]:
model = smf.mixedlm(
    'N12M_Pseudomonas_aeruginosa ~ abundance + Pseudomonas_Culture + CS_Pseudomonas_aeruginosa', 
    merged_data,
    groups=merged_data["pwCF_ID"] 
)
result = model.fit()

# Print the model summary
print(result.summary())


In [ ]:
if False:
    model = smf.mixedlm(
        'abundance ~ CS_Pseudomonas_aeruginosa + taxa_functions', 
        merged_data,
        groups=merged_data["pwCF_ID"] 
    )
    result = model.fit()

    # Print the model summary
    print(result.summary())


## Write the results

In [ ]:
coefficients = pd.DataFrame({
    "Variable": result.params.index,
    "Estimate": result.params.values,
    "Std Error": result.bse.values,
    "P-value": result.pvalues.values
})
coefficients.to_csv(os.path.join('lmm', f'{taxa}_model_coefficients.tsv'), sep="\t", index=False)

with open(os.path.join('lmm', f'{taxa}_model_results.tsv'), 'w') as f:
    f.write(result.summary().as_text())

In [ ]:
"""
# Extract fixed effect coefficients and confidence intervals
params = result.fe_params  # Fixed effect coefficients
conf_int = result.conf_int()  # Confidence intervals
conf_int.columns = ['lower', 'upper']

# Combine coefficients and confidence intervals
coefficients = pd.DataFrame({
    'coef': params,
    'lower': conf_int['lower'],
    'upper': conf_int['upper']
})

# Plot the coefficients with error bars
plt.figure(figsize=(8, 6))
plt.errorbar(coefficients.index, coefficients['coef'], 
             yerr=(coefficients['coef'] - coefficients['lower'], coefficients['upper'] - coefficients['coef']), 
             fmt='o', capsize=5)
plt.axhline(0, color='gray', linestyle='--', linewidth=1)
plt.xticks(rotation=45)
plt.title("Fixed Effect Coefficients with Confidence Intervals")
plt.ylabel("Coefficient Value")
plt.xlabel("Predictor")
plt.tight_layout()
plt.show()
"""